In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from quspin.basis import spin_basis_1d
from quspin.operators import quantum_operator


plt.rcParams.update({
    "text.usetex": True,
})
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Times New Roman'
plt.rcParams['mathtext.fallback'] = 'stix'
plt.rcParams['font.family'] ='Times New Roman'
plt.style.use('seaborn-v0_8-deep')
prop_cycle = plt.rcParams['axes.prop_cycle']
dcolors = prop_cycle.by_key()['color']

# Model

The O'brian-Fendley model is much like the normal Majorana-Hubbard chain but with a slightly modified interaction term:

\begin{align*}
H_{OF} =& -it\sum_j \gamma_j \gamma_{j+1} -g \sum_j \gamma_{j-2}\gamma_{j-1}\gamma_{j+1}\gamma_{j+2}\\
=& t \sum_j \left(\sigma_j^x +\sigma_j^z \sigma_{j+1}^z\right)
+g \sum_j\left(\sigma_j^x \sigma_{j+1}^x \sigma_{j+2}^z + \sigma_j^z \sigma_{j+1}^z \sigma_{j+2}^z\right)
\end{align*}

In [ ]:
def Hof(t, g, basis):
    """
    t, g are couplings
    basis is quspin spinless_fermion_basis_1d
    """
    
    L = basis.N
    
    x_lst = [[-2*t, i] for i in range(L)]
    
    zz_lst = [[-2*t, i, i+1] for i in range(L-1)]
    
    xzz_lst = [[g, i, i+1, i+2] for i in range(L-2)]
    zzx_lst = [[g, i, i+1, i+2] for i in range(L-2)]

    zz_lst += [[-2*t, L-1, 0]]
    
    xzz_lst += [[g, L-2, L-1, 0],
                [g, L-1, 0, 1]]
    
    zzx_lst += [[g, L-2, L-1, 0],
                [g, L-1, 0, 1]]
    
    H = quantum_operator({'static': [['x', x_lst],
                                     ['zz', zz_lst],
                                     ['xzz', xzz_lst],
                                     ['zzx', zzx_lst],
                                    ]}, 
                         basis=basis, check_symm=False, check_herm=False)
    return H

In [ ]:
L = 4
basis = spin_basis_1d(L, pauli=1) # pauli matrix, not spin 1/2 (matches iTensor "S=1/2")
t = 1
g = 1
H = Hof(t, g, basis)
e, v = H.eigh()
print(e[:10]) # +L*(t**2+g**2)/g)